# Cygwin

For window 10 users

1. Download cygwin - ([Download](https://www.cygwin.com/install.html))
    - I downloaded `32 bits` because the book was based on it
2. Install cygwin **(In case of versions, I installed the most recent ones!)**
    - binutils (source & binary)
    - bison (binary)
    - flex (binary)
    - gcc-core (source & binary)
    - gcc-g++ (binary)
    - libiconv (binary)
    - libtool (binary)
    - make (binary)
    - patchutils (binary)
    - libgmp-devel (binary)
    - libmpfr-devel (binary)
3. Set environment path
    - C:\cygwin\bin
    - C:\cygwin\usr\cross\bin
4. Test

In [ ]:
# make this with test.c file
#include <stdio.h>

int main(int argc, char **argv)
{
    printf("Hello, world\n");
    return 0;
}

In [ ]:
gcc -m32 -o test32 test.c
gcc -m64 -o test64 test.c

If you downloaded `32 bits` it won't work at `-m64`.<br>
If you downloaded as `64 bits` it won't work at `m32`.

5. Make cross compiler for 32/64 bits
    - open the `cygwin terminal`

In [ ]:
cd /usr/src/binutils-2.34-1.src/

# unzip
tar -xvf binutils-gdb-2.34.tar.bz2
cd binutils-gdb

export TARGET=x86_64-pc-linux
export PREFIX=/usr/cross

./configure --target=$TARGET
            --prefix $PREFIX
            --enable-64-bit-bfd
            --disable-shared
            --disable-nls
            
make configure-host

If following error appears it means you don't have `texinfo` package

In [ ]:
/usr/src/binutils-2.34-1.src/binutils-gdb/missing: line 81: makeinfo: command not found
WARNING: 'makeinfo' is missing on your system.
         You should only need it if you modified a '.texi' file, or
         any other file indirectly affecting the aspect of the manual.
         You might want to install the Texinfo package:
         <http://www.gnu.org/software/texinfo/>
         The spurious makeinfo call might also be the consequence of
         using a buggy 'make' (AIX, DU, IRIX), in which case you might
         want to install GNU make:
         <http://www.gnu.org/software/make/>
make[2]: *** [Makefile:541: bfd.info] Error 127
make[2]: Leaving directory '/usr/src/binutils-2.34-1.src/binutils-gdb/bfd/doc'
make[1]: *** [Makefile:1654: info-recursive] Error 1
make[1]: Leaving directory '/usr/src/binutils-2.34-1.src/binutils-gdb/bfd'
make: *** [Makefile:2753: all-bfd] Error 2


***The following steps should be done in `powershell administrator`*** unless it will cause an error!

For window10 user let's install `chocolately`
- **Install chocolately**
- https://chocolatey.org/

The `texinfo` is included to the following package! Install it!
- Install **GnuWin 0.6.3**
- https://chocolatey.org/packages/GnuWin/0.6.3#files

In [ ]:
choco install gnuwin --version=0.6.3

When you finished it let's go back to installing `binutils`

In [ ]:
# If there was error previously we type again. If there was not then skip this line
make configure-host

make LDFLAGS="-all-static"

`-all-static` problem occurs

In [ ]:
  GEN    init.c
    CXX    init.o
  CXXLD  gdb.exe
g++: error: unrecognized command line option ‘-all-static’; did you mean ‘--static’?
make[2]: *** [Makefile:1841: gdb.exe] Error 1
make[2]: Leaving directory '/usr/src/binutils-2.34-1.src/binutils-gdb/gdb'
make[1]: *** [Makefile:11280: all-gdb] Error 2
make[1]: Leaving directory '/usr/src/binutils-2.34-1.src/binutils-gdb'
make: *** [Makefile:857: all] Error 2


This problem is discussed on here https://github.com/JuliaPackaging/BinaryBuilder.jl/issues/372

and the suggested solution is 

In [ ]:
./configure LDFLAGS=-static; make LDFLAGS=-all-static

modified a bit with adding some options

In [ ]:
./configure --target=$TARGET --prefix=$PREFIX --enable-64-bit-bfd --disable-shared --disable-nls LDFLAGS=-static

Another error occurs because we added `LDFLAGS=-static`

In [ ]:
# We configured again so do this again
$  make configure-host

Configuring in ./libiberty
configure: loading cache ./config.cache
configure: error: `LDFLAGS' has changed since the previous run:
configure:   former value:  ` -Wl,--stack,12582912'
configure:   current value: ` -static -Wl,--stack,12582912'
configure: error: in `/usr/src/binutils-2.34-1.src/binutils-gdb/libiberty':
configure: error: changes in the environment can compromise the build
configure: error: run `make distclean' and/or `rm ./config.cache' and start over
make: *** [Makefile:8346: configure-libiberty] Error 1

So we had to remove `config.cache` files from `libiberty` as error message says

In [ ]:
make distclean

cd libiberty
rm ./config.cache
cd ..

./configure --target=$TARGET --prefix=$PREFIX --enable-64-bit-bfd --disable-shared --disable-nls LDFLAGS=-static
make LDFLAGS=-all-static

This cause another error

In [ ]:
Configuring in ./intl
configure: loading cache ./config.cache
configure: error: `LDFLAGS' has changed since the previous run:
configure:   former value:  ` -Wl,--stack,12582912'
configure:   current value: ` -static -Wl,--stack,12582912'
configure: error: in `/usr/src/binutils-2.34-1.src/binutils-gdb/intl':


Maybe this is not the end. Rather suffering from cache I decided to start over

In [ ]:
# remove the binutils-gdb folder
cd ..
rm -rf binutils-gdb
tar -xvf binutils-gdb-2.34.tar.bz2

cd binutills-gdb
./configure --target=$TARGET --prefix=$PREFIX --enable-64-bit-bfd --disable-shared --disable-nls LDFLAGS=-static
make configure-host
make LDFLAGS=--static

It still fails. Doing again. This time do it directly with static

In [ ]:
./configure --target=$TARGET --prefix=$PREFIX --enable-64-bit-bfd --disable-shared --disable-nls
make configure-host
make LDFLAGS=--static

In [ ]:
/usr/lib/gcc/i686-pc-cygwin/9.3.0/../../../../i686-pc-cygwin/bin/ld: cannot find -lmpfr
/usr/lib/gcc/i686-pc-cygwin/9.3.0/../../../../i686-pc-cygwin/bin/ld: cannot find -lgmp

looking at the following page ran the following command

https://stackoverflow.com/questions/30128309/cygwin-gcc-error-while-loading-shared-libraries

In [ ]:
$ cygcheck /usr/lib/gcc/i686-pc-cygwin/9.3.0/cc1.exe

# Following results
C:\cygwin\lib\gcc\i686-pc-cygwin\9.3.0\cc1.exe
  C:\cygwin\bin\cygwin1.dll
    C:\WINDOWS\system32\KERNEL32.dll
      C:\WINDOWS\system32\ntdll.dll
      C:\WINDOWS\system32\KERNELBASE.dll
  C:\cygwin\bin\cyggmp-10.dll
  C:\cygwin\bin\cygiconv-2.dll
  C:\cygwin\bin\cygintl-8.dll
  C:\cygwin\bin\cygisl-22.dll
  C:\cygwin\bin\cygmpc-3.dll
    C:\cygwin\bin\cygmpfr-6.dll
      C:\cygwin\bin\cyggcc_s-1.dll
  C:\cygwin\bin\cygz.dll

Well Not successful this time.. So I decided to reference the `OSdev wiki with GCC Cross-Compiler`. **Now let's start over!** I removed `C:\cygwin` to start totally again!

**Windows Users**

https://wiki.osdev.org/GCC_Cross-Compiler#Windows_Users

Windows users need to set up a Unix-like enviroment such as MinGW or Cygwin. It may well be worth looking into systems such as Linux and see if they fit your needs, as you commonly use a lot of Unix-like tools in operating systems development and this is much easier from a Unix-like operating system. If you have just installed the basic Cygwin package, you have to run the setup.exe again and install the following packages: `GCC, G++, Make, Flex, Bison, Diffutils, libintl-devel, libgmp-devel, libmpfr-devel, libmpc-devel, Texinfo`

**Download cygwin (32 bits)**
- **binutils (source & binary)**
- Diffutils (binary)
- bison (binary)
- flex (binary)
- **gcc-core (source & binary)**
- gcc-g++ (binary)
- libiconv (binary)
- libtool (binary)
- make (binary)
- patchutils (binary)
- libintl-devel (binary)
- libgmp-devel (binary)
- libmpfr-devel (binary)
- libmpc-devel (binary)
- Texinfo (binary)


In [ ]:
export TARGET=x86_64-pc-linux
export PREFIX=/usr/cross

# add --disable-werror & --with-sysroot referenced by OSdev wiki
./configure --target=$TARGET --prefix=$PREFIX --with-sysroot --disable-werror --enable-64-bit-bfd --disable-shared --disable-nls
make configure-host
make LDFLAGS=--static

Still the same problem happens..

In [ ]:
/usr/lib/gcc/i686-pc-cygwin/9.3.0/../../../../i686-pc-cygwin/bin/ld: cannot find -lmpfr
/usr/lib/gcc/i686-pc-cygwin/9.3.0/../../../../i686-pc-cygwin/bin/ld: cannot find -lgmp

https://stackoverflow.com/questions/42436350/cygwin-linker-doesnt-find-shared-library

This stackoverflow tells `cygwin` expects to have library in `.dll` extension.

I went to `/usr/lib`and **found that `mpfr` and `gmp` has no static library only dynamic**
- `libmpfr.dll.a`
- `libgmp.dll.a` 

we need

- `libmpfr.a`
- `libgmp.a`

This is not quiet right but just simply created symbolic link

- `ln -s libmpfr.dll.a libmpfr.a`
- `ln -s libgmp.dll.a libgmp.a` 

In [ ]:
make LDFLAGS=--static

In [ ]:
# still an error

make[2]: Entering directory '/usr/src/binutils-2.34-1.src/binutils-gdb/gdb'
  CXXLD  gdb.exe
/usr/lib/gcc/i686-pc-cygwin/9.3.0/../../../../i686-pc-cygwin/bin/ld: disasm-selftests.o: in function `Z28_initialize_disasm_selftestsv':
/usr/src/binutils-2.34-1.src/binutils-gdb/gdb/disasm-selftests.c:218: undefined reference to `selftests::register_test_foreach_arch(std::string const&, void (*)(gdbarch*))'
/usr/lib/gcc/i686-pc-cygwin/9.3.0/../../../../i686-pc-cygwin/bin/ld: /usr/src/binutils-2.34-1.sr
/usr/src/binutils-2.34-1.src/binutils-gdb/gdb/dwarf2-frame.c:2376: undefined reference to `selftests::register_test_foreach_c/binutils-gdb/gdb/disasm-selftests.c:220: undefined reference to `selftests::register_test_foreach_arch(std::string const&, void (*)(gdbarch*))'
/usr/lib/gcc/i686-pc-cygwin/9.3.0/../../../../i686-pc-cygwin/bin/ld: dwarf2-frame.o: in function `Z24_initialize_dwarf2_framev':arch(std::string const&, void (*)(gdbarch*))'
/usr/lib/gcc/i686-pc-cygwin/9.3.0/../../../../i686-pc-cygwin/bin/ld: gdbarch-selftests.o: in function `Z29_initialize_gdbarch_selftestsv':
/usr/src/binutils-2.34-1.src/binutils-gdb/gdb/gdbarch-selftests.c:168: undefined reference to `selftests::register_test_foreach_arch(std::string const&, void (*)(gdbarch*))'
/usr/lib/gcc/i686-pc-cygwin/9.3.0/../../../../i686-pc-cygwin/bin/ld: regcache.o: in function `Z20_initialize_regcachev':
/usr/src/binutils-2.34-1.src/binutils-gdb/gdb/regcache.c:1874: undefined reference to `selftests::register_test_foreach_arch(std::string const&, void (*)(gdbarch*))'
/usr/lib/gcc/i686-pc-cygwin/9.3.0/../../../../i686-pc-cygwin/bin/ld: regcache.o:/usr/src/binutils-2.34-1.src/binutils-gdb/gdb/regcache.c:1876: more undefined references to `selftests::register_test_foreach_arch(std::string const&, void (*)(gdbarch*))' follow
/usr/lib/gcc/i686-pc-cygwin/9.3.0/../../../../i686-pc-cygwin/bin/ld: ../gdbsupport/libgdbsupport.a(selftest.o): in function `ZN9selftests9run_testsEPKc':
/usr/src/binutils-2.34-1.src/binutils-gdb/gdbsupport/selftest.c:96: undefined reference to `selftests::reset()'
collect2: error: ld returned 1 exit status


In [ ]:
# Tried to find this register_test_foreach_arch

$ grep -rn register_test_foreach_arch

# results
ChangeLog-2017:7671:    (register_test_foreach_arch): Add name parameter.  Call
ChangeLog-2017:7676:    * selftest-arch.h (register_test_foreach_arch): Add name
ChangeLog-2017:9831:    selftests::register_test_foreach_arch.
disasm-selftests.c:218:  selftests::register_test_foreach_arch ("print_one_insn",
disasm-selftests.c:220:  selftests::register_test_foreach_arch ("memory_error",
Binary file disasm-selftests.o matches
dwarf2-frame.c:2376:  selftests::register_test_foreach_arch ("execute_cfa_program",
Binary file dwarf2-frame.o matches
gdbarch-selftests.c:168:  selftests::register_test_foreach_arch ("register_to_value",
Binary file gdbarch-selftests.o matches
regcache.c:1874:  selftests::register_test_foreach_arch ("regcache::cooked_read_test",
regcache.c:1876:  selftests::register_test_foreach_arch ("regcache::cooked_write_test",
Binary file regcache.o matches
                                                         
# This seems the place where I was finding for                                                         
selftest-arch.c:92:register_test_foreach_arch (const std::string &name,
selftest-arch.h:27:  register_test_foreach_arch (const std::string &name,
                                                 
unittests/unpack-selftests.c:60:  selftests::register_test_foreach_arch


In [ ]:
# selftest-arch.c:92
void
register_test_foreach_arch (const std::string &name,
                            self_test_foreach_arch_function *function)
{
  register_test (name, new gdbarch_selftest (function));
}

# selftest-arch.h:27
namespace selftests
{
extern void
  register_test_foreach_arch (const std::string &name,
                              self_test_foreach_arch_function *function);
}

# disasm-selftests.c

#include "defs.h"
#include "disasm.h"

#if GDB_SELF_TEST
#include "gdbsupport/selftest.h"
#include "selftest-arch.h"
#include "gdbarch.h"

namespace selftests {

/* Test disassembly of one instruction.  */
----- blah blah blah
} // namespace selftests
#endif /* GDB_SELF_TEST */

void _initialize_disasm_selftests ();
void
_initialize_disasm_selftests ()
{
#if GDB_SELF_TEST
  selftests::register_test_foreach_arch ("print_one_insn",
                                         selftests::print_one_insn_test);
  selftests::register_test_foreach_arch ("memory_error",
                                         selftests::memory_error_test);
#endif
}


Let's look at the object file

In [ ]:
# -C option is same as c++filt
$ nm -C disasm-selftests.o

00000000 b .bss
00000000 d .data
00000000 N .debug_abbrev
00000000 N .debug_aranges
00000000 N .debug_info
00000000 N .debug_line
00000000 N .debug_loc
00000000 N .debug_ranges
00000000 N .debug_str
00000000 r .eh_frame
00000000 d .gcc_except_table
00000000 r .rdata
00000000 r .rdata$_ZTI13gdb_exception
00000000 r .rdata$_ZTI19gdb_exception_error
00000000 r .rdata$_ZTS13gdb_exception
00000000 r .rdata$_ZTS19gdb_exception_error
00000000 r .rdata$zzz
00000000 t .text
00000000 t .text.unlikely
         U _Unwind_Resume
         U gdbarch_bfd_arch_info(gdbarch*)
00000340 T _initialize_disasm_selftests()
00000090 t _initialize_disasm_selftests() [clone .cold]
         U gdbarch_breakpoint_kind_from_pc(gdbarch*, unsigned long long*)
         U gdbarch_sw_breakpoint_from_kind(gdbarch*, int, int*)
         U error(char const*, ...)
         U gdb_disassembler::print_insn(unsigned long long, int*)
         U gdb_disassembler::gdb_disassembler(gdbarch*, ui_file*, int (*)(unsigned long long, unsigned char*, unsigned int, disassemble_info*))
         
         # We can see this option is Undefined..
         U selftests::register_test_foreach_arch(std::string const&, void (*)(gdbarch*))
000000e0 t selftests::memory_error_test(gdbarch*)
00000000 t selftests::memory_error_test(gdbarch*) [clone .cold]
00000160 t selftests::print_one_insn_test(gdbarch*)
0000006c t selftests::print_one_insn_test(gdbarch*) [clone .cold]
         U std::string::_Rep::_M_destroy(std::allocator<char> const&)
000000b0 t std::string::_Rep::_M_dispose(std::allocator<char> const&) [clone .part.0]
         U std::string::_Rep::_S_empty_rep_storage
         U std::basic_string<char, std::char_traits<char>, std::allocator<char> >::basic_string(char const*, std::allocator<char> const&)
00000000 R typeinfo for gdb_exception
00000000 R typeinfo for gdb_exception_error
00000000 R typeinfo name for gdb_exception
00000000 R typeinfo name for gdb_exception_error
         U vtable for __cxxabiv1::__class_type_info
         U vtable for __cxxabiv1::__si_class_type_info
000000a0 t selftests::memory_error_test(gdbarch*)::gdb_disassembler_test::read_memory(unsigned long long, unsigned char*, unsigned int, disassemble_info*)
00000180 r selftests::print_one_insn_test(gdbarch*)::xstormy16_insn
00000184 r selftests::print_one_insn_test(gdbarch*)::arm_insn
00000188 r selftests::print_one_insn_test(gdbarch*)::bfin_insn
00000182 r selftests::print_one_insn_test(gdbarch*)::s390_insn
00000000 t selftests::print_one_insn_test(gdbarch*)::gdb_disassembler_test::read_memory(unsigned long long, unsigned char*, unsigned int, disassemble_info*)
         U __cxa_begin_catch
         U __cxa_end_catch
         U __gxx_personality_v0
         U __umoddi3
         U null_stream


In [ ]:
U selftests::register_test_foreach_arch(std::string const&, void (*)(gdbarch*))